In [84]:
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.utils import class_weight

In [37]:
def load_individual_instance_to_2d_array(filename, needed_columns):
    df = pd.read_csv(filename)
    for col in needed_columns:
         if not (col in df.columns):
            df[col] = 0.0
    # Ensure all the columns are in the correct order!
    rearranged = df[needed_columns]  
    return df.to_numpy()

In [63]:
def train_list_to_train_3d_array(train_list):
    np3d_a = np.dstack(train_list)
    np3d = np.moveaxis(np3d_a, 2,0)
    return np3d

In [65]:
def create_combined_data(base_dir, filenames, needed_columns):
    combined_data_m = map(lambda file: load_individual_instance_to_2d_array(base_dir + "/" + file, needed_columns), filenames)
    combined_data = train_list_to_train_3d_array(list(combined_data_m))
    return combined_data

In [74]:
def single_metric_value(m, divisor):
    f = np.floor(m/divisor)
    if (f < 0):
        return 0.0
    else:
        return f

In [68]:
def metrics_map_to_class(metrics, divisor):
      return np.array([single_metric_value(v,divisor) for v in metrics])

In [168]:
def create_model_for_prediction():
    tf.random.set_seed(42)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=[500, 10]))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(5000, activation="selu"))
    model.add(tf.keras.layers.Dense(500, activation="selu"))
    model.add(tf.keras.layers.Dense(11, activation="softmax"))
    return model

In [130]:
def set_class_weights(train_class):
    class_weights_np = class_weight.compute_class_weight('balanced', classes = np.unique(train_class), y = train_class)
    print("class_weights_np=", class_weights_np)
    class_weights_dict = {}
    for c in range(0,class_weights_np.size):
        class_weights_dict[c] = class_weights_np[c]
    print("class_weights_dict=", class_weights_dict)
    return class_weights_dict

In [167]:
def mlp_keras_classifier(csv_directory_train, csv_directory_test, train_mfile, test_mfile, needed_columns, metric_name, metric_divisor, num_epochs, batch_size):
    files_train = sorted(os.listdir(csv_directory_train))
    files_test = sorted(os.listdir(csv_directory_test))
    train_data = create_combined_data(csv_directory_train, files_train, needed_columns)
    test_data = create_combined_data(csv_directory_test, files_test, needed_columns)
    train_metrics = pd.read_csv(train_mfile)[metric_name]
    test_metrics = pd.read_csv(test_mfile)[metric_name]
    train_class = metrics_map_to_class(train_metrics, metric_divisor)
    test_class = metrics_map_to_class(test_metrics, metric_divisor)

    print("Max train class: ", np.max(train_class))
    
    print("Train data shape:", train_data.shape)
    print("Train metrics shape:", train_metrics.shape)
    print("Test data shape:", test_data.shape)
    print("Test metrics shape:", test_metrics.shape)
    print("Train class shape:", train_class.shape)
    print("Test metrics shape:", test_class.shape)

    model = create_model_for_prediction()
    print(model.summary())

    sgd_n = tf.keras.optimizers.SGD(nesterov=True, learning_rate=0.02)
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=sgd_n, metrics=["accuracy"])
    class_weights = set_class_weights(train_class)
    print(class_weights)
    history = model.fit(train_data, train_class, epochs=num_epochs, validation_data=(test_data, test_class), class_weight=class_weights, batch_size=batch_size)
    predictions = model.predict(test_data)
    return [model, test_class, predictions]
    

In [165]:
base_dir = "./temp-data-400-objectsdelivered-augmented/"
metric_name = "M1_countObjectsDelivered"

needed_columns = ['dosAttackTrigger', 'distortBaseScanPMB2', 'distortBaseScanOmni', 'distortBaseScanTiago', 'packetLossLaserScanOmni', 'packetLossLaserScanPMB2', 'packetLossLaserScanTiago', 'delayBaseScanOmni', 'delayBaseScanPMB2', 'delayBaseScanTiago1']
csv_directory_train = base_dir + "/train"
csv_directory_test = base_dir + "/test"
mfile_train = base_dir + "metrics-train.csv"
mfile_test = base_dir + "metrics-test.csv"
train_metrics = pd.read_csv(mfile_train)
metric_divisor = 3

files_train = sorted(os.listdir(csv_directory_train))
files_train
train_data = create_combined_data(csv_directory_train, files_train, needed_columns)

In [169]:
num_epochs = 300
batch_size=512
mlp_keras_classifier(csv_directory_train, csv_directory_test, mfile_train, mfile_test, needed_columns, metric_name, metric_divisor, num_epochs, batch_size)

Max train class:  10.0
Train data shape: (4724, 500, 10)
Train metrics shape: (4724,)
Test data shape: (1588, 500, 10)
Test metrics shape: (1588,)
Train class shape: (4724,)
Test metrics shape: (1588,)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_23 (Flatten)            │ (None, 5000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 5000)           │    25,005,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 500)            │     2,500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 11)             │         5,511 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,511,011 (104.95 MB)

 Trainable params: 27,511,011 (104.95 MB)

 Non-trainable params: 0 (0.00 B)

None
class_weights_np= [  3.80967742   0.65248619   0.18926282   0.46313725   1.93606557
   7.87333333  29.525       39.36666667  19.68333333 118.1       ]
class_weights_dict= {0: 3.8096774193548386, 1: 0.6524861878453039, 2: 0.1892628205128205, 3: 0.4631372549019608, 4: 1.9360655737704917, 5: 7.873333333333333, 6: 29.525, 7: 39.36666666666667, 8: 19.683333333333334, 9: 118.1}
{0: 3.8096774193548386, 1: 0.6524861878453039, 2: 0.1892628205128205, 3: 0.4631372549019608, 4: 1.9360655737704917, 5: 7.873333333333333, 6: 29.525, 7: 39.36666666666667, 8: 19.683333333333334, 9: 118.1}
Epoch 1/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.0410 - loss: 2.8489 - val_accuracy: 0.0642 - val_loss: 2.4420
Epoch 2/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.0543 - loss: 1.8065 - val_accuracy: 0.0743 - val_loss: 2.3641
Epoch 3/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.0873 - loss: 1.5825 - val_accuracy: 0.0989 - val_loss: 2.2971
Epoch 4/300
10/10 ━━━━━━━━━━━━━━

[<Sequential name=sequential_23, built=True>,
 array([1., 1., 1., ..., 2., 2., 2.]),
 array([[1.7017905e-03, 5.1367164e-01, 1.7495708e-01, ..., 1.9058045e-02,
         2.9559518e-05, 4.3874668e-04],
        [8.0214599e-03, 2.1801688e-01, 5.4697019e-01, ..., 2.1532910e-06,
         1.8491683e-05, 1.3540344e-05],
        [1.7017905e-03, 5.1367164e-01, 1.7495708e-01, ..., 1.9058045e-02,
         2.9559518e-05, 4.3874668e-04],
        ...,
        [7.7361947e-06, 1.0451199e-01, 2.9833841e-01, ..., 5.9033587e-06,
         9.2871301e-07, 5.4729684e-05],
        [6.0456246e-01, 4.3999653e-02, 1.4180575e-01, ..., 1.3060104e-04,
         2.1648232e-04, 4.3004987e-04],
        [6.0456246e-01, 4.3999653e-02, 1.4180575e-01, ..., 1.3060104e-04,
         2.1648232e-04, 4.3004987e-04]], dtype=float32)]

In [144]:
res = Out[143]

In [145]:
res

[<Sequential name=sequential_20, built=True>,
 array([1., 1., 1., ..., 2., 2., 2.]),
 array([[2.0634164e-01, 2.0293707e-01, 2.3189253e-01, ..., 1.4089298e-01,
         2.8062395e-03, 1.4728548e-04],
        [1.2239524e-01, 2.9997924e-01, 4.4350168e-01, ..., 1.3866481e-04,
         3.1666720e-04, 2.9734823e-05],
        [2.0634164e-01, 2.0293707e-01, 2.3189253e-01, ..., 1.4089298e-01,
         2.8062395e-03, 1.4728548e-04],
        ...,
        [2.5154131e-03, 1.9057545e-01, 1.6677156e-01, ..., 8.6867582e-04,
         8.2091326e-05, 1.9093066e-04],
        [1.6656312e-01, 1.7070270e-01, 1.8303491e-01, ..., 1.0564726e-02,
         1.4190877e-02, 5.0038458e-03],
        [1.6656312e-01, 1.7070270e-01, 1.8303491e-01, ..., 1.0564726e-02,
         1.4190877e-02, 5.0038458e-03]], dtype=float32)]

In [146]:
res[0]

<Sequential name=sequential_20, built=True>

In [160]:
[res[1][9], np.argmax(res[2][9])]

[2.0, 2]